In [12]:
import matplotlib.pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
from numpy import pi
from mpmath.calculus.optimization import findroot
import numpy as np
from sympy import erfc, exp
from mpmath import *
import deloc
from tkinter import *

In [13]:

# Constants for calculations
# meV for energies
# cm for lengths

q_initial = float(input("Enter the initial value for domain: "))

q_final = float(input("Enter the final value for domain: "))

q_step = float(input("Enter the wave number step: "))

m = float(input("Enter the cavity order m: "))

Ec = float(input("Enter the Ec: "))

E0 = float(input("Enter the E0: "))

Lc = float(input("Enter the cavity length: "))

Delta = float(input("Enter the rabi energy: "))

Sigma = float(input("Enter the energetic disorder: "))

Gamma_Ex = float(input("Enter the molecular disorder: "))

Gamma_L = float(input("Enter the cavity leakage: "))

a = float(input("Enter the intermolecular distance: "))

q_domain = np.arange(q_initial, q_final, q_step)


In [14]:
q_inel_array_lp = np.empty(len(q_domain), dtype = float)

for i in range(len(q_domain)):
    q_inel_array_lp[i] = deloc.q_inel_lp(m, q_domain[i], Ec, E0, Lc, Delta, Gamma_Ex, Gamma_L)

q_inel_array_up = np.empty(len(q_domain), dtype = float)

for i in range(len(q_domain)):
    q_inel_array_up[i] = deloc.q_inel_up(m, q_domain[i], Ec, E0, Lc, Delta, Gamma_Ex, Gamma_L)

# Calculate the differences between consecutive elements
diff_arr_inel_lp = np.diff(np.sign(q_inel_array_lp))

# Find the indices where the sign changes
indices_inel_lp = np.where(diff_arr_inel_lp != 0)[0] + 1

if len(indices_inel_lp) == 1:
    q_inel_lp_min = q_domain[indices_inel_lp[0]]  # Assign the element to q_min
    q_inel_lp_max = q_final
else:
    q_inel_lp_min = q_domain[indices_inel_lp[0]]
    q_inel_lp_max = q_domain[indices_inel_lp[1]]  # Assign the second element to q_max

# Calculate the differences between consecutive elements
diff_arr_inel_up = np.diff(np.sign(q_inel_array_up))

# Find the indices where the sign changes
indices_inel_up = np.where(diff_arr_inel_up != 0)[0] + 1

if len(indices_inel_up) == 1:
    q_inel_up_min = q_domain[indices_inel_up[0]]  # Assign the element to q_min
    q_inel_up_max = q_final
else:
    q_inel_up_min = q_domain[indices_inel_up[0]]
    q_inel_up_max = q_domain[indices_inel_up[1]]  # Assign the second element to q_max

In [15]:
q_fluc_array_lp = np.empty(len(q_domain), dtype = float)

for i in range(len(q_domain)):
    q_fluc_array_lp[i] = deloc.q_fluc_lp(m, q_domain[i], Ec, E0, Lc, Delta, a)

q_fluc_array_up = np.empty(len(q_domain), dtype = float)

for i in range(len(q_domain)):
    q_fluc_array_up[i] = deloc.q_fluc_up(m, q_domain[i], Ec, E0, Lc, Delta, a)

# Calculate the differences between consecutive elements
diff_arr_fluc_lp = np.diff(np.sign(q_fluc_array_lp))

# Find the indices where the sign changes
indices_fluc_lp = np.where(diff_arr_fluc_lp != 0)[0] + 1

if len(indices_fluc_lp) == 1:
    q_fluc_lp_min = q_domain[indices_fluc_lp[0]]  # Assign the element to q_min
    q_fluc_lp_max = q_final
else:
    q_fluc_lp_min = q_domain[indices_fluc_lp[0]]
    q_fluc_lp_max = indices_fluc_lp[1]  # Assign the second element to q_max

# Calculate the differences between consecutive elements
diff_arr_fluc_up = np.diff(np.sign(q_fluc_array_up))

# Find the indices where the sign changes
indices_fluc_up = np.where(diff_arr_fluc_up != 0)[0] + 1

if len(indices_fluc_up) == 1:
    q_fluc_up_min = q_domain[indices_fluc_up[0]]  # Assign the element to q_min
    q_fluc_up_max = q_final
else:
    q_fluc_up_min = q_domain[indices_fluc_up[0]]
    q_fluc_up_max = q_domain[indices_fluc_up[1]]  # Assign the second element to q_max

In [19]:
# ----------------- Energy Broadening due to Resonant Scattering ---------------------

# fadeeva function
# exp*erfc
def f(x): return exp(-(x ** 2)) * erfc(-1j*x)

def gamma_res_lp(m, x, Ec, E0, Lc, Delta, Sigma, imag_part_lp):
    """Calculate lower polariton energy broadening due to resonant scattering."""

    B = ((Delta / Sigma) ** 2) * np.sqrt(np.pi)

    # Prompt the user for the real and imaginary parts of the initial guess
    real_part_lp = deloc.energy_lp_zero(m, x, Ec, E0, Lc, Delta)
    #imag_part_lp = float(input("Enter the imaginary part of the initial guess LP: ") or x*deloc.group_velocity_lp(x,Ec, E0, Lc, Delta))

    # Create the initial guess as a complex number
    initial_guess_lp = complex(real_part_lp, imag_part_lp)
    t = findroot(lambda y: 1j * f((y - E0) / Sigma) * B * Sigma + y - Ec * np.sqrt(1 + ((x * Lc) / np.pi) ** 2),
                 initial_guess_lp)
    return t.imag

# Initialize the arrays with the q_domain shape
gamma_res_array_lp = np.empty_like(q_domain)
l_res_array_lp = np.empty_like(q_domain)
q_res_array_lp = np.empty_like(q_domain)

# Calculate the arrays element-wise
for i in range(len(q_domain)-1):
    gamma_res_array_lp[0] = gamma_res_lp(m, q_domain[0], Ec, E0, Lc, Delta, Sigma, 0)
    gamma_res_array_lp[i+1] = gamma_res_lp(m, q_domain[i+1], Ec, E0, Lc, Delta, Sigma, gamma_res_array_lp[i])


# ----------------- Mean Free Path of Resonant Scattering ---------------------

l_res_array_lp = deloc.group_velocity_lp(m, q_domain, Ec, E0, Lc, Delta)/ abs(gamma_res_array_lp)

# ----------------- Resonant Boundary Points ---------------------

q_res_array_lp = l_res_array_lp - 1/q_domain

# Calculate the differences between consecutive elements
diff_arr_res_lp = np.diff(np.sign(q_res_array_lp))

# Find the indices where the sign changes
indices_res_lp = np.where(diff_arr_res_lp != 0)[0] + 1

if len(indices_res_lp) == 1:
    q_res_lp_min = q_domain[indices_res_lp[0]]  # Assign the element to q_min
    q_res_lp_max = q_final
else:
    q_res_lp_min = q_domain[indices_res_lp[0]]
    q_res_lp_max = q_domain[indices_res_lp[1]]  # Assign the second element to q_max


In [21]:
# ----------------- Energy Broadening due to Resonant Scattering UP ---------------------

# fadeeva function
# exp*erfc
def f(x): return exp(-(x ** 2)) * erfc(-1j*x)

def gamma_res_up(m, x, Ec, E0, Lc, Delta, Sigma, imag_part_up):
    """Calculate lower polariton energy broadening due to resonant scattering."""

    B = ((Delta / Sigma) ** 2) * np.sqrt(np.pi)

    # Prompt the user for the real and imaginary parts of the initial guess
    real_part_up = deloc.energy_up_zero(m, x, Ec, E0, Lc, Delta)
    #imag_part_lp = float(input("Enter the imaginary part of the initial guess LP: ") or x*deloc.group_velocity_lp(x,Ec, E0, Lc, Delta))

    # Create the initial guess as a complex number
    initial_guess_up = complex(real_part_up, imag_part_up)
    t = findroot(lambda y: 1j * f((y - E0) / Sigma) * B * Sigma + y - Ec * np.sqrt(1 + ((x * Lc) / np.pi) ** 2),
                 initial_guess_up)
    return t.imag

# Initialize the arrays with the q_domain shape
gamma_res_array_up = np.empty_like(q_domain)
l_res_array_up = np.empty_like(q_domain)
q_res_array_up = np.empty_like(q_domain)

# Calculate the arrays element-wise
for i in range(len(q_domain)-1):
    gamma_res_array_up[0] = gamma_res_up(m, q_domain[0], Ec, E0, Lc, Delta, Sigma, 0)
    gamma_res_array_up[i+1] = gamma_res_up(m, q_domain[i+1], Ec, E0, Lc, Delta, Sigma, gamma_res_array_up[i])
    # Check if the current element is too small
    if abs(gamma_res_array_up[i+1]) < 1e-6:
        # Make the rest of the array zero
        gamma_res_array_up[i+2:] = 1e-10
        # Exit the loop
        break

# ----------------- Mean Free Path of Resonant Scattering ---------------------

l_res_array_up = deloc.group_velocity_up(m, q_domain, Ec, E0, Lc, Delta)/ abs(gamma_res_array_up)

# ----------------- Resonant Boundary Points ---------------------

for i in range(len(q_domain)):
    q_res_array_lp[i] = l_res_array_lp[i] - 1 / q_domain[i]

# Calculate the differences between consecutive elements
diff_arr_res_up = np.diff(np.sign(q_res_array_up))

# Find the indices where the sign changes
indices_res_up = np.where(diff_arr_res_up != 0)[0] + 1

q_res_up_min = q_domain[indices_res_up[0]]  # Assign the element to q_min


In [23]:
# Compare three scatterings to determine the minimum boundary for UP

q_min_up = max(q_fluc_up_min, q_inel_up_min, q_res_up_min)

# Compare three scatterings to determine the minimum boundary for LP

q_min_lp = max(q_fluc_lp_min, q_inel_lp_min, q_res_lp_min)

# Compare three scatterings to determine the maximum boundary for LP

q_max_lp = min( q_fluc_lp_max, q_inel_lp_max, q_res_lp_max)

q_min_lp, q_max_lp, q_min_up

(6401.0, 20701.0, 5101.0)